## **Link** **Prediction** **and** **Evaluation** 

In [ ]:
import pandas as pd
df1 = pd.read_csv('/content/ent_labels.tsv',header=None)
df = (pd.read_csv('/content/ent_embedding.tsv',sep='\t',header=None))
df.index = df1[0].tolist()

In [ ]:
f = open("kg1-train.txt","r")
data = f.read()
data = data.replace('\t1\t', '\t')
data = data.replace('\t2\t','\t')
data = data.replace('\t3\t','\t')
data = data.replace('\t4\t', '\t')
f = open("edge_list-train.txt","w")
f.write(data)

243235

In [ ]:
f = open("kg1-test.txt","r")
data = f.read()
data = data.replace('\t1\t', '\t')
data = data.replace('\t2\t','\t')
data = data.replace('\t3\t','\t')
data = data.replace('\t4\t', '\t')
f = open("edge_list-test.txt","w")
f.write(data)

28660

In [ ]:
f = open("kg1-valid.txt","r")
data = f.read()
data = data.replace('\t1\t', '\t')
data = data.replace('\t2\t','\t')
data = data.replace('\t3\t','\t')
data = data.replace('\t4\t', '\t')
f = open("edge_list-valid.txt","w")
f.write(data)

14325

In [ ]:
import networkx as nx
adj = open("edge_list-train.txt","r")
lines = adj.readlines()
g_train = nx.Graph()
for l in lines: 
  l = l.strip('\n')
  a = l.split('\t')
  g_train.add_edge(a[0],a[1])
print(len(g_train.edges()))

18553


In [ ]:
import networkx as nx
adj = open("edge_list-test.txt","r")
lines = adj.readlines()
g_test = nx.Graph()
for l in lines: 
  l = l.strip('\n')
  a = l.split('\t')
  g_test.add_edge(a[0],a[1])
print(len(g_test.edges()))

2545


In [ ]:
import networkx as nx
adj = open("edge_list-valid.txt","r")
lines = adj.readlines()
g_valid = nx.Graph()
for l in lines: 
  l = l.strip('\n')
  a = l.split('\t')
  g_valid.add_edge(a[0],a[1])
print(len(g_valid.edges()))

1278


In [ ]:
import numpy as np
import random


def edges_All(edges):
  edges_all = []
  edge = []
  for e in edges :
    edge.append([int(e[0]),int(e[1])])
    edges_all.append([int(e[0]),int(e[1])])
    edges_all.append([int(e[1]),int(e[0])])
  return edge,edges_all

def mask_test_edges():
    # Function to build test set with 10% positive links

    val_edges = g_valid.edges()
    train_edges = g_train.edges()
    test_edges = g_test.edges()
    test_edges,test_edges_all = edges_All(test_edges)
    train_edges, train_edges_all = edges_All(train_edges)
    val_edges, val_edges_all = edges_All(val_edges)
    edges_all = []
    edges_all.append(test_edges_all)
    edges_all.append(train_edges_all)
    edges_all.append(val_edges_all)    
    user = df1[0].tolist()
    product = df1[0].tolist()
    test_edges_false = []
    while len(test_edges_false) < len(test_edges):
        idx_i = random.choice(user)
        idx_j = random.choice(product)
        if idx_i == idx_j:
            continue
        if [idx_i, idx_j] in edges_all:
            continue
        if test_edges_false:
            if [idx_j, idx_i] in test_edges_false:
                continue
            if [idx_i, idx_j] in test_edges_false:
                continue
        test_edges_false.append([idx_i, idx_j])

    val_edges_false = []
    while len(val_edges_false) < len(val_edges):
        idx_i = random.choice(user)
        idx_j = random.choice(product)
        if idx_i == idx_j:
            continue
        if [idx_i, idx_j] in edges_all:
            continue
        if [idx_i, idx_j] in test_edges_false:
            continue
        if [idx_j, idx_i] in test_edges_false:
            continue
        if val_edges_false:
            if [idx_j, idx_i] in val_edges_false:
                continue
            if [idx_i, idx_j] in val_edges_false:
                continue
        val_edges_false.append([idx_i, idx_j])

    train_edges_false = []
    while len(train_edges_false) < len(train_edges):
        idx_i = random.choice(user)
        idx_j = random.choice(product)
        if idx_i == idx_j:
            continue
        if [idx_i, idx_j] in edges_all:
            continue
        if train_edges_false:
            if [idx_j, idx_i] in train_edges_false:
                continue
            if [idx_i, idx_j] in train_edges_false:
                continue     

        train_edges_false.append([idx_i,idx_j])

    return train_edges, train_edges_false, val_edges, val_edges_false, test_edges, test_edges_false


In [ ]:
import numpy as np
train_edges, train_edges_false, val_edges, val_edges_false, test_edges, test_edges_false = mask_test_edges()

In [ ]:
def get_edge_embeddings(edge_list):
    embs = []
    for edge in edge_list:
        node1 = edge[0]
        node2 = edge[1]
        emb1 = list(df.loc[node1])
        emb2 = list(df.loc[node2])
        edge_emb = np.multiply(emb1, emb2)
        embs.append(edge_emb)
    embs = np.array(embs)
    return embs

In [ ]:
pos_train_edge_embs = get_edge_embeddings(train_edges)
neg_train_edge_embs = get_edge_embeddings(train_edges_false)
train_edge_embs = np.concatenate([pos_train_edge_embs, neg_train_edge_embs])

# Create train-set edge labels: 1 = real edge, 0 = false edge
train_edge_labels = np.concatenate([np.ones(len(train_edges)), np.zeros(len(train_edges_false))])

# Val-set edge embeddings, labels
pos_val_edge_embs = get_edge_embeddings(val_edges)
neg_val_edge_embs = get_edge_embeddings(val_edges_false)
val_edge_embs = np.concatenate([pos_val_edge_embs, neg_val_edge_embs])
val_edge_labels = np.concatenate([np.ones(len(val_edges)), np.zeros(len(val_edges_false))])

# Test-set edge embeddings, labels
pos_test_edge_embs = get_edge_embeddings(test_edges)
neg_test_edge_embs = get_edge_embeddings(test_edges_false)
test_edge_embs = np.concatenate([pos_test_edge_embs, neg_test_edge_embs])

# Create val-set edge labels: 1 = real edge, 0 = false edge
test_edge_labels = np.concatenate([np.ones(len(test_edges)), np.zeros(len(test_edges_false))])

Logistic Regression

In [ ]:
# Train logistic regression classifier on train-set edge embeddings
from sklearn.linear_model import LogisticRegression
edge_classifier = LogisticRegression(random_state=0)
edge_classifier.fit(train_edge_embs, train_edge_labels)

LogisticRegression(random_state=0)

In [ ]:
import scipy.sparse as sp
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
# Predicted edge scores: probability of being of class "1" (real edge)

In [ ]:
val_preds = edge_classifier.predict_proba(val_edge_embs)[:, 1]
val_roc = roc_auc_score(val_edge_labels, val_preds)
val_ap = average_precision_score(val_edge_labels, val_preds)

In [ ]:
test_preds = edge_classifier.predict_proba(test_edge_embs)[:, 1]
test_roc = roc_auc_score(test_edge_labels, test_preds)
test_ap = average_precision_score(test_edge_labels, test_preds)

In [ ]:
print('Validation ROC score: ', str(val_roc))
print('Validation AP score: ', str(val_ap))
print('Test ROC score: ', str(test_roc))
print('Test AP score: ', str(test_ap))

Validation ROC score:  0.8016836018720566
Validation AP score:  0.8660913704457754
Test ROC score:  0.7888529070059171
Test AP score:  0.8580575901424574


Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
edge_classifier = RandomForestClassifier()
edge_classifier.fit(train_edge_embs, train_edge_labels)
import scipy.sparse as sp
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
# Predicted edge scores: probability of being of class "1" (real edge)
val_preds = edge_classifier.predict(val_edge_embs)
val_roc = roc_auc_score(val_edge_labels, val_preds)
val_ap = average_precision_score(val_edge_labels, val_preds)
val_preds = edge_classifier.predict(val_edge_embs)
val_roc = roc_auc_score(val_edge_labels, val_preds)
val_ap = average_precision_score(val_edge_labels, val_preds)
test_preds = edge_classifier.predict(test_edge_embs)
test_roc = roc_auc_score(test_edge_labels, test_preds)
test_ap = average_precision_score(test_edge_labels, test_preds)
print('Validation ROC score: ', str(val_roc))
print('Validation AP score: ', str(val_ap))
print('Test ROC score: ', str(test_roc))
print('Test AP score: ', str(test_ap))

Validation ROC score:  0.8163080635431329
Validation AP score:  0.7903412219823391
Test ROC score:  0.822061191626409
Test AP score:  0.7960667348599169


SVM

In [ ]:
from sklearn.svm import SVC
edge_classifier = SVC()
edge_classifier.fit(train_edge_embs, train_edge_labels)
import scipy.sparse as sp
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
# Predicted edge scores: probability of being of class "1" (real edge)
val_preds = edge_classifier.predict(val_edge_embs)
val_roc = roc_auc_score(val_edge_labels, val_preds)
val_ap = average_precision_score(val_edge_labels, val_preds)
val_preds = edge_classifier.predict(val_edge_embs)
val_roc = roc_auc_score(val_edge_labels, val_preds)
val_ap = average_precision_score(val_edge_labels, val_preds)
test_preds = edge_classifier.predict(test_edge_embs)
test_roc = roc_auc_score(test_edge_labels, test_preds)
test_ap = average_precision_score(test_edge_labels, test_preds)
print('Validation ROC score: ', str(val_roc))
print('Validation AP score: ', str(val_ap))
print('Test ROC score: ', str(test_roc))
print('Test AP score: ', str(test_ap))

Validation ROC score:  0.7956967625175949
Validation AP score:  0.7769906709178778
Test ROC score:  0.7976046698872785
Test AP score:  0.778407196336376


K-Nearest Neighbour 

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
edge_classifier = KNeighborsClassifier()
edge_classifier.fit(train_edge_embs, train_edge_labels)
import scipy.sparse as sp
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
# Predicted edge scores: probability of being of class "1" (real edge)
val_preds = edge_classifier.predict(val_edge_embs)
val_roc = roc_auc_score(val_edge_labels, val_preds)
val_ap = average_precision_score(val_edge_labels, val_preds)
val_preds = edge_classifier.predict(val_edge_embs)
val_roc = roc_auc_score(val_edge_labels, val_preds)
val_ap = average_precision_score(val_edge_labels, val_preds)
test_preds = edge_classifier.predict(test_edge_embs)
test_roc = roc_auc_score(test_edge_labels, test_preds)
test_ap = average_precision_score(test_edge_labels, test_preds)
print('Validation ROC score: ', str(val_roc))
print('Validation AP score: ', str(val_ap))
print('Test ROC score: ', str(test_roc))
print('Test AP score: ', str(test_ap))

Validation ROC score:  0.7960989342449226
Validation AP score:  0.7865079239899266
Test ROC score:  0.7971014492753623
Test AP score:  0.7871479873211309
